## Create from manual terms

In [29]:
file_path = r'C:\Users\Usuario\Documents\JupyterFolder\unimi_files\IR\glossary.txt' # Replace with the path to your text file
with open(file_path, 'r') as file: lines = file.readlines()
glossary = [line.strip() for line in lines]
# print(glossary[:10])

file_path = r'C:\Users\Usuario\Documents\JupyterFolder\unimi_files\IR\target_topics.txt'
with open(file_path, 'r') as file: lines = file.readlines()
targets = [line.strip() for line in lines]
# print(targets[:10])

In [30]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize

import numpy as np
np.random.seed(400)

stemmer = SnowballStemmer("english")
lemmatizer = WordNetLemmatizer()

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def lemmatize(token):
    return lemmatizer.lemmatize(token, pos='v')

def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 2:
            result.append(lemmatize_stemming(token))
    return result

def save_reference(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 2:
            result.append(lemmatize(token))
    return result

glossary_processed = [preprocess(doc) for doc in glossary]
glossary_reference = [save_reference(doc) for doc in glossary]

targets_processed = [preprocess(doc) for doc in targets]
targets_reference = [save_reference(doc) for doc in targets]

processed_docs = glossary_processed + targets_processed
reference_docs = glossary_reference + targets_reference

print(processed_docs[:3])
print()

# print(len(processed_docs))
# print(len(reference_docs))
# print()

stem_ref = [stem for doc in processed_docs for stem in doc]
lemm_ref = [lemma for doc in reference_docs for lemma in doc]

print(reference_docs[:3])

[['abstract'], ['abstract', 'strategi', 'game'], ['accessori']]

[['abstract'], ['abstract', 'strategy', 'game'], ['accessory']]


In [31]:
def find_element_in_lists(a, b, target):
    try:
        index = a.index(target)
        return b[index]
    except ValueError:
        return None

In [32]:
from gensim.corpora import Dictionary

dictionary = gensim.corpora.Dictionary(processed_docs)  
dictionary.save('dictionary_file')
dictionary = Dictionary.load('dictionary_file')
for token, idx in dictionary.token2id.items(): print(find_element_in_lists(stem_ref, lemm_ref, token), idx)

abstract 0
game 1
strategy 2
accessory 3
act 4
action 5
allowance 6
point 7
selection 8
alpha 9
player 10
ameritrash 11
analysis 12
paralysis 13
area 14
control 15
enclosure 16
impulse 17
movement 18
arg 19
auction 20
bbg 21
balance 22
beer 23
pretzels 24
bet 25
bgg 26
patron 27
bid 28
bits 29
blind 30
block 31
wargame 32
bluff 33
bpa 34
break 35
bsw 36
card 37
draft 38
ccg 39
cdg 40
chit 41
counter 42
chrome 43
closer 44
coin 45
collectible 46
computational 47
cooperative 48
core 49
group 50
crayon 51
rail 52
crib 53
sheet 54
crt 55
build 56
deck 57
designer 58
dexterity 59
dice 60
die 61
fest 62
roll 63
downtime 64
crawl 65
dungeon 66
drm 67
dry 68
dudes 69
map 70
economic 71
end 72
euro 73
eurogame 74
eurodude 75
eurotrash 76
expansion 77
experience 78
family 79
factor 80
fart 81
fiddly 82
filler 83
flgs 84
friendly 85
tie 86
abbreviations 87
gameid 88
gamer 89
gamey 90
gateway 91
gbg 92
geek 93
geekbadge 94
geekbuddy 95
geekcoin 96
geekgold 97
geeklists 98
geekmod 99
week 100
germa

## Extract from corpus

In [59]:
import pandas as pd
pd.set_option('display.max_columns', None)
import json
import os
import numpy as np
import ast

path_original_data = r'C:\Users\Usuario\Documents\JupyterFolder\unimi_files\IR'
df = pd.read_csv(os.path.join(path_original_data,'post_processed_comment_data_demo.csv'), low_memory=False)
string_list = df['gensim_comment_verbs'].tolist()
comments_list = [ast.literal_eval(s) for s in string_list]

# comments_list = []
# for s in string_list: comments_list.append(ast.literal_eval(s))
# print(comments_list[:2])  

Original size:  37392

Trimmed size:  1740 

Trimmed size:  1589 

In [60]:
def find_unique_elements(list1, list2):
    unique_elements_list1 = list(set(list1) - set(list2))
    unique_elements_list2 = list(set(list2) - set(list1))
    unique_elements = unique_elements_list1 + unique_elements_list2
    return unique_elements

def write_words_to_file(word_list, filename):
    with open(filename, 'w') as file:
        for word in word_list:
            file.write(word + '\n')

In [61]:
auxiliar = gensim.corpora.Dictionary(comments_list) 
print('Original size: ',len(auxiliar))
list_a = [token for token, idx in auxiliar.token2id.items()]
    
auxiliar.filter_extremes(no_below=100, no_above=0.5, keep_n= None)
print('Trimmed size: ',len(auxiliar), '\n')
list_b = [token for token, idx in auxiliar.token2id.items()]



import nltk
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from gensim import corpora

lemmatizer = WordNetLemmatizer()
verbs = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']

tokens = [token for token in auxiliar.values()]
pos_tags = nltk.pos_tag(tokens)

filtered_tokens = [token for token, pos_tag in zip(tokens, pos_tags) if pos_tag[1] not in verbs]
filtered_dictionary = corpora.Dictionary()
filtered_dictionary.doc2bow(filtered_tokens, allow_update=True)

print('Original size: ',len(auxiliar))
print('Trimmed size: ',len(filtered_dictionary), '\n')
list_c = [token for token, idx in filtered_dictionary.token2id.items()]

Original size:  34764
Trimmed size:  1436 


Original size:  1436 

Trimmed size:  1325 

Dictionary(1325 unique tokens: ['abil', 'abl', 'absolut', 'abstract', 'access']...)


In [62]:
# for x in find_unique_elements(list_a, list_b): print(x)

In [63]:
# for x in find_unique_elements(list_b, list_c): print(x)

In [64]:
from gensim.corpora import Dictionary

auxiliar.save('auxiliar_dictionary_file')
auxiliar = Dictionary.load('auxiliar_dictionary_file')
# for token, idx in auxiliar.token2id.items(): print(token, idx)
    
write_words_to_file([token for token, idx in auxiliar.token2id.items()], 'auxiliary_list.txt')

## Fine tune the dictionary

In [65]:
def read_words_from_file(filename):
    word_list = []
    with open(filename, 'r') as file:
        for line in file:
            word = line.strip()
            word_list.append(word)
    return word_list

In [66]:
print('Original length: ', len(dictionary))
dictionary.merge_with(auxiliar)
print('New length: ', len(dictionary), '\n')

Original length:  1865
New length:  1867 



In [69]:
filename = 'words_to_remove.txt'
words_to_remove = read_words_from_file(filename)
print(words_to_remove, '\n')
print(dictionary)
word_ids = [dictionary.token2id[word] for word in words_to_remove if word in dictionary.token2id]
dictionary.filter_tokens(bad_ids=word_ids)
dictionary.compactify()
# for word_id in word_ids: dictionary.token2id.popitem(word_id)
print(dictionary, '\n')

from gensim.corpora import Dictionary

dictionary.save('final_dictionary_file')
dictionary = Dictionary.load('final_dictionary_file')
# for token, idx in auxiliar.token2id.items(): print(token, idx)

['time', 'card', 'great', 'like', 'good', 'board', 'fun', 'game', 'play', 'player'] 

Dictionary(1860 unique tokens: ['abstract', 'strategi', 'accessori', 'act', 'action']...)
Dictionary(1858 unique tokens: ['abstract', 'strategi', 'accessori', 'act', 'action']...) 



# Reference

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8534395/

https://jmlr.csail.mit.edu/papers/volume21/20-079/20-079.pdf